In [2]:
import pandas as pd
import re

In [3]:
def remove_extra(s):
    return(re.sub(r'\(.*\)|\{.*\}|\'|\"', "", s).strip())

In [4]:
#701 actors

In [135]:
def get_non_duplicated(artist_dataframe):
    artist_counts = artist_dataframe[['movies','artist']].groupby('artist',as_index=False).agg(['count'])
    artist_counts.columns = [''.join(col).strip() for col in artist_counts.columns.values]
    artist_counts2 = artist_counts.reset_index()
    print('Only',artist_counts2['artist'].count(),' unique names')
    
    artist_counts2  = artist_counts2.loc[artist_counts2['moviescount'] >1]
    duplicate_artists = list(artist_counts2['artist'].values)
    #print(len(duplicate_artists),' repeated names')
    duplicated_artist_df  = artist_dataframe.loc[artist_dataframe['artist'].isin(duplicate_artists)]
    non_duplicated_artist_df  = artist_dataframe.loc[~artist_dataframe['artist'].isin(duplicate_artists)]
    print(non_duplicated_artist_df['movies'].count(),' non repeated names')
    fixed_duplicated_artist_df = duplicated_artist_df.groupby('artist').agg({'movies':'sum'})
    fixed_duplicated_artist_df['movies'] = fixed_duplicated_artist_df['movies'].apply(lambda x: list(set(x)))
    print(fixed_duplicated_artist_df['movies'].count(),' responsible for duplicates')
    non_duplicated_artist_df.append(fixed_duplicated_artist_df)
    return non_duplicated_artist_df

    

In [136]:
def get_artist_data(file_name):
    artist_list = []
    artist_movies_list = []
    i = 0
    with open(file_name,'rb') as f:
        lines = f.readlines()
        for line in lines:
            i+=1
            if(i%100000==0):
                print(i)
            line= line.decode('utf-8','ignore').encode("utf-8")
            line_split = line.decode().split('\t\t')
            artist = line_split[0]
            movies = []
            if(len(line_split)>1):
                movies = set(map(remove_extra, line_split[1:]))
                movies =[m for m in movies if(len(m))>0]
            artist_list.append(artist)
            artist_movies_list.append(list(movies))     
    print(len(artist_list)," Lines found")
    artist_dataframe = pd.DataFrame(list(zip(artist_list, artist_movies_list)),columns=['artist','movies'])
    return get_non_duplicated(artist_dataframe)
    



In [137]:
non_duplicated_actor_df = get_artist_data('data/actor_movies.txt')

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2167653  Lines found
Only 2166817  unique names
2166116  non repeated names
701  responsible for duplicates


In [138]:
non_duplicated_actress_df = get_artist_data('data/actress_movies.txt')

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1182813  Lines found
Only 1182404  unique names
1182084  non repeated names
320  responsible for duplicates


In [169]:
non_duplicated_artist_df = get_non_duplicated(non_duplicated_actor_df.append(non_duplicated_actress_df))
non_duplicated_artist_df['count'] = non_duplicated_artist_df['movies'].apply(lambda x: len(x))
non_duplicated_artist_df.head()

Only 3348112  unique names
3348024  non repeated names
88  responsible for duplicates


,artist,movies,count
0,"$, Claw",[],0
1,"$, Homo","[Suuri illusioni, Nykytaiteen museo]",2
2,"$, Steve",[],0
3,"$hort, Too","[Townbiz, Menace II Society, Rhyme & Reason, S...",12
4,"$lim, Bee Moe","[For Thy Love 2, Fatherhood 101]",2


In [174]:
non_duplicated_artist_df.count()

artist    3348024
movies    3348024
count     3348024
dtype: int64

In [176]:
final_df.count()

artist    100783
movies    100783
count     100783
dtype: int64

In [170]:
final_df = non_duplicated_artist_df.loc[non_duplicated_artist_df['count'] >10]

In [177]:
artists = final_df['artist'].values
len(artists)

100783

In [ ]:
num_artists = len(artists)#len(artists)
edge_tuple=[]
for i in range(num_artists):
    print("Evaluating node ",i)
    i_movies = list(final_df['movies'].loc[final_df['artist'] == artists[i]])[0]
    den = len(i_movies)
    for j in range(num_artists):
        if i ==j:
            continue
        j_movies = final_df['movies'].loc[final_df['artist'] == artists[j]]
        j_movies = list(j_movies)[0]
        #print(j_movies)
        weight = len(set(i_movies)&set(j_movies))*1.0/den
        if(weight>0):
            edge_tuple.append((i,j,weight))
        

Evaluating node  0


In [182]:
edge_tuple

[(2, 98, 0.02564102564102564), (9, 22, 0.045454545454545456)]